In [1]:
import numpy as np
import csv, json
import pandas as pd
from datetime import datetime

In [2]:
with open('Data/MStanleyStocks/MS.csv', "rt", encoding="utf8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    # Converting the csv file reader to a lists 
    data_list = list(spamreader)
# Separating header from the data
header = data_list[0] 
data_list = data_list[1:] 
data_list = np.asarray(data_list)

# Selecting date and close value for each day
selected_data = data_list[:, [0, 2, 3, 4, 5]]
print(selected_data)

df = pd.DataFrame(data=selected_data[0:,1:],
             index=selected_data[0:,0],
                                columns=['High', 'Low', 'close', 'Open'],
                                        dtype='float64')
df.head()

[['2006-11-20' '80.000000' '78.260002' '79.309998' '55.404018']
 ['2006-11-21' '79.750000' '79.099998' '79.230003' '55.348129']
 ['2006-11-22' '80.000000' '78.769997' '79.199997' '55.327171']
 ..., 
 ['2017-11-24' '49.389999' '49.040001' '49.060001' '49.060001']
 ['2017-11-27' '49.320000' '48.709999' '49.000000' '49.000000']
 ['2017-11-28' '50.529999' '48.930000' '50.400002' '50.400002']]


,High,Low,close,Open
2006-11-20,80.000000,78.260002,79.309998,55.404018
2006-11-21,79.750000,79.099998,79.230003,55.348129
2006-11-22,80.000000,78.769997,79.199997,55.327171
2006-11-24,79.250000,78.150002,78.949997,55.152531
2006-11-27,78.830002,76.089996,76.250000,53.266369


In [3]:
date_format = ["%Y-%m-%dT%H:%M:%SZ", "%Y-%m-%dT%H:%M:%S+%f"]
def try_parsing_date(text):
    for fmt in date_format:
        #return datetime.strptime(text, fmt)
        try:
            return datetime.strptime(text, fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [4]:
# Adding missing dates to the dataframe
df1 = df
idx = pd.date_range('12-29-2006', '12-31-2016')
df1.index = pd.DatetimeIndex(df1.index)
df1 = df1.reindex(idx, fill_value=np.NaN)
interpolated_df = df1.interpolate()
interpolated_df["articles"] = ""
interpolated_df.head() # gives 3651 count

,High,Low,close,Open,articles
2006-12-29,82.000,81.250,81.430000,56.885002,
2006-12-30,82.064,81.164,81.468001,56.911545,
2006-12-31,82.128,81.078,81.506001,56.938089,
2007-01-01,82.192,80.992,81.544002,56.964632,
2007-01-02,82.256,80.906,81.582002,56.991176,


To find out the JSON structure of the data, 
Check this website: http://jsonviewer.stack.hu/

In [5]:
current_article_str = '' 
current_date = '2016-01-01'
count_total_articles = 0
count_articles_filtered = 0
count_attribute_error = 0
count_main_not_exist = 0 
dict_keys = ['pub_date', 'headline']
articles_dict = dict.fromkeys(dict_keys)
type_of_material_list = ['blog', 'brief', 'news', 'editorial', 'op-ed', 'list','analysis']
section_name_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health']
news_desk_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health', 'foreign']
years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [6]:
# for year in years:
#     for month in months:
file_str = 'Data/NYTimes/Stock2007-01.json'
with open(file_str) as data_file:
    NYTimes_data = json.load(data_file)
    print(NYTimes_data["response"]["docs"][:][0])
    print("-----------------------------------------------------------------------------------------------------------------------------")
    print(NYTimes_data["response"]["docs"][:][0]['type_of_material'].lower())
    
    print("-----------------------------------------------------------------------------------------------------------------------------")
    print (NYTimes_data["response"]["docs"][:][0]['headline'])
    print("-----------------------------------------------------------------------------------------------------------------------------")
    print (NYTimes_data["response"]["docs"][:][0]['pub_date'])
    print("-----------------------------------------------------------------------------------------------------------------------------")
    print (NYTimes_data["response"]["docs"][:][0]['headline']['main'])
    date = try_parsing_date(NYTimes_data["response"]["docs"][:][0]['pub_date'])
    print("-----------------------------------------------------------------------------------------------------------------------------")
    print (date)
    


{'web_url': 'https://fifthdown.blogs.nytimes.com/2007/01/01/the-view-from-boston/', 'snippet': "The Jets hardly had the most challenging schedule this season, but they did beat the Patriots at New England, where they'll return Sunday for a playoff game. It was the Jets' most impressive victory of the season. A seemingly gleeful Ron Borges of...", 'lead_paragraph': "The Jets hardly had the most challenging schedule this season, but they did beat the Patriots at New England, where they'll return Sunday for a playoff game.", 'abstract': "The Jets hardly had the most challenging schedule this season, but they did beat the Patriots at New England, where they'll return Sunday for a playoff game. It was the Jets' most impressive victory of the season. A seemingly gleeful Ron Borges of The Boston Globe described it this way. (Borges has been a fierce [...]", 'print_page': None, 'blog': [], 'source': 'The New York Times', 'multimedia': [], 'headline': {'main': 'The View From Boston', 'kicker': 

In [7]:
print("Started")
for year in years:
    for month in months:
#         print("Year, Month", year, month)
        file_str = 'Data/NYTimes/Stock' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str) as data_file:    
            NYTimes_data = json.load(data_file)
        count_total_articles = count_total_articles + len(NYTimes_data["response"]["docs"][:])
        for i in range(len(NYTimes_data["response"]["docs"][:])):
            try:
                if any(substring in NYTimes_data["response"]["docs"][:][i]['type_of_material'].lower() for substring in type_of_material_list):
                    if any(substring in NYTimes_data["response"]["docs"][:][i]['section_name'].lower() for substring in section_name_list):
                        #count += 1
                        count_articles_filtered += 1
                        #print 'i: ' + str(i)
                        articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                        articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                        #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph
                        date = try_parsing_date(articles_dict['pub_date'])
                        #print 'article_dict: ' + articles_dict['headline']
                        if date == current_date:
                            current_article_str = current_article_str + '. ' + articles_dict['headline']
                        else:  
                            interpolated_df.set_value(current_date, 'articles', interpolated_df.loc[current_date, 'articles'] + '. ' + current_article_str)
                            current_date = date
                            #interpolated_df.set_value(date, 'articles', current_article_str)
                            #print str(date) + current_article_str
                            current_article_str = articles_dict['headline']
                        # For last condition in a year
                        if (date == current_date) and (i == len(NYTimes_data["response"]["docs"][:]) - 1): 
                            interpolated_df.set_value(date, 'articles', current_article_str)   
                        
             #Exception for section_name or type_of_material absent
            except AttributeError:
                #print 'attribute error'
                #print NYTimes_data["response"]["docs"][:][i]
                count_attribute_error += 1
                # If article matches news_desk_list if none section_name found
                try:
                    if any(substring in NYTimes_data["response"]["docs"][:][i]['news_desk'].lower() for substring in news_desk_list):
                            #count += 1
                            count_articles_filtered += 1
                            #print 'i: ' + str(i)
                            articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                            articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                            #articles_dict['headline'] = articles_dict['lead_paragraph'] # Selecting lead_paragraph
                            date = try_parsing_date(articles_dict['pub_date'])
                            #print 'article_dict: ' + articles_dict['headline']
                            if date == current_date:
                                current_article_str = current_article_str + '. ' + articles_dict['headline']
                            else:  
                                interpolated_df.set_value(current_date, 'articles', interpolated_df.loc[current_date, 'articles'] + '. ' + current_article_str)
                                current_date = date
                                #interpolated_df.set_value(date, 'articles', current_article_str)
                                #print str(date) + current_article_str
                                current_article_str = articles_dict['headline']
                            # For last condition in a year
                            if (date == current_date) and (i == len(NYTimes_data["response"]["docs"][:]) - 1): 
                                interpolated_df.set_value(date, 'articles', current_article_str)   
                
                except AttributeError:
                    pass
                pass
            except KeyError:
                print ('key error')
#                 print (NYTimes_data["response"]["docs"][:][i])
                count_main_not_exist += 1
                pass   
            except TypeError:
                print ("type error")
                #print NYTimes_data["response"]["docs"][:][i]
                count_main_not_exist += 1
                pass
            
print("Finished Successfully")
         

Started
type error
key error
type error
key error
type error
type error
key error
type error
type error
key error
key error
key error
key error
key error
key error
type error
Finished Successfully


In [8]:
print (count_articles_filtered) 
print (count_total_articles)                     
print (count_main_not_exist)
interpolated_df.head(10)

461738
1248084
16


,High,Low,close,Open,articles
2006-12-29,82.000000,81.250000,81.430000,56.885002,
2006-12-30,82.064000,81.164000,81.468001,56.911545,
2006-12-31,82.128000,81.078000,81.506001,56.938089,
2007-01-01,82.192000,80.992000,81.544002,56.964632,. Estimates of Iraqi Civilian Deaths. Romania ...
2007-01-02,82.256000,80.906000,81.582002,56.991176,". For Dodd, Wall Street Looms Large. Ford's Lo..."
2007-01-03,82.320000,80.820000,81.620003,57.017719,". Ethics Changes Proposed for House Trips, K S..."
2007-01-04,82.400002,80.500000,81.910004,57.220310,. I Feel Bad About My Face. Bush Recycles the ...
2007-01-05,81.589996,80.360001,80.860001,56.486801,. Macworld Bingo. Anti-Surge Protests Against ...
2007-01-06,81.599998,80.253334,81.023333,56.600902,. In da Car at Dakar. The Macworld-C.E.S. Conf...
2007-01-07,81.610001,80.146668,81.186666,56.715004,. BitTorrent Comes to the Television. LG&#8217...


In [9]:
print("Started")
for date, row in interpolated_df.T.iteritems():
    if len(interpolated_df.loc[date, 'articles']) <= 400:
        month = date.month
        year = date.year
#         print(year, month)
        file_str = 'Data/NYTimes/Stock' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str) as data_file:
            NYTimes_data = json.load(data_file)
        count_total_articles = count_total_articles + len(NYTimes_data["response"]["docs"][:])
        interpolated_df.set_value(date.strftime('%Y-%m-%d'), 'articles', '')
        for i in range(len(NYTimes_data["response"]["docs"][:])):
            try:
                articles_dict = { your_key: NYTimes_data["response"]["docs"][:][i][your_key] for your_key in dict_keys }
                articles_dict['headline'] = articles_dict['headline']['main'] # Selecting just 'main' from headline
                pub_date = try_parsing_date(articles_dict['pub_date'])
                if date.strftime('%Y-%m-%d') == pub_date:
                    interpolated_df.set_value(pub_date, 'articles', interpolated_df.loc[pub_date, 'articles'] + '. ' + articles_dict['headline'])
            except KeyError:
                print ('key error')
                pass
            except TypeError:
                print ("type error")
                pass
print("Finished Successfully")

Started
type error
type error
type error
type error
type error
type error
type error
type error
key error
key error
key error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
key error
key error
key error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
type error
key error
key error
key error
type error
type error
type error
type error
type error
type error
type error
type error
type error
key error
type error
key error
key error
type error
type error
key error
key error
Finished Successfully


In [10]:
interpolated_df.head(10)

,High,Low,close,Open,articles
2006-12-29,82.000000,81.250000,81.430000,56.885002,. The Marais. Looking Back at World Events. Vo...
2006-12-30,82.064000,81.164000,81.468001,56.911545,. Win-a-Trip!. Lessons From Saddam. Evaluating...
2006-12-31,82.128000,81.078000,81.506001,56.938089,. Warm Memories of Gerald Ford on a Chilly Day...
2007-01-01,82.192000,80.992000,81.544002,56.964632,. Estimates of Iraqi Civilian Deaths. Romania ...
2007-01-02,82.256000,80.906000,81.582002,56.991176,". For Dodd, Wall Street Looms Large. Ford's Lo..."
2007-01-03,82.320000,80.820000,81.620003,57.017719,". Ethics Changes Proposed for House Trips, K S..."
2007-01-04,82.400002,80.500000,81.910004,57.220310,. I Feel Bad About My Face. Bush Recycles the ...
2007-01-05,81.589996,80.360001,80.860001,56.486801,. Macworld Bingo. Anti-Surge Protests Against ...
2007-01-06,81.599998,80.253334,81.023333,56.600902,. In da Car at Dakar. The Macworld-C.E.S. Conf...
2007-01-07,81.610001,80.146668,81.186666,56.715004,. BitTorrent Comes to the Television. LG&#8217...


In [11]:

# Saving the data as pickle file
interpolated_df.to_pickle('Data/NYT.pkl')  


# Save pandas frame in csv form
interpolated_df.to_csv('Data/NYT.csv',
                       sep=',', encoding='utf-8')

print("Saved successfully as NYT.csv")


Saved successfully as NYT.csv


MACD: The Moving Average Convergence/Divergence oscillator (MACD) is one of the simplest and most effective momentum indicators available. The MACD turns two trend-following indicators, moving averages, into a momentum oscillator by subtracting the longer moving average from the shorter moving average.

Stochastics oscillator: The Stochastic Oscillator is a momentum indicator that shows the location of the close relative to the high-low range over a set number of periods.

Average True Range: Is an indicator to measure the volalitility (NOT price direction). The largest of:
Method A: Current High less the current Low
Method B: Current High less the previous Close (absolute value)
Method C: Current Low less the previous Close (absolute value)

In [12]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [13]:
def MACD(df,period1,period2,periodSignal):
    EMA1 = pd.DataFrame.ewm(df,span=period1).mean()
    EMA2 = pd.DataFrame.ewm(df,span=period2).mean()
    MACD = EMA1-EMA2
    
    Signal = pd.DataFrame.ewm(MACD,periodSignal).mean()
    
    Histogram = MACD-Signal
    
    return Histogram

def stochastics_oscillator(df,period):
    l, h = pd.DataFrame.rolling(df, period).min(), pd.DataFrame.rolling(df, period).max()
    k = 100 * (df - l) / (h - l)
    return k

def ATR(df,period):
    '''
    Method A: Current High less the current Low
    '''
    df['H-L'] = abs(df['High']-df['Low'])
    df['H-PC'] = abs(df['High']-df['close'].shift(1))
    df['L-PC'] = abs(df['Low']-df['close'].shift(1))
    TR = df[['H-L','H-PC','L-PC']].max(axis=1)
    return TR.to_frame()

In [14]:
# df= pd.read_csv('Data/NYT.csv',usecols=[1])
df = pd.read_csv('Data/NYT.csv',usecols=[1,2,3,4,5])
df = df.iloc[::-1]
dfPrices = pd.read_csv('Data/NYT.csv',usecols=[3])
dfPrices = dfPrices.iloc[::-1]
dfPrices.head()

,close
3655,42.250000
3654,42.250000
3653,42.150002
3652,42.619999
3651,43.119999


In [15]:
macd = MACD(dfPrices.iloc[len(dfPrices.index)-60:len(dfPrices.index)],12,26,9)

In [16]:
stochastics = stochastics_oscillator(dfPrices.iloc[len(dfPrices.index)-60:len(dfPrices.index)],14)

In [17]:
atr = ATR(df.iloc[len(df.index)-60:len(df.index)],14)

In [18]:
macd = MACD(dfPrices,12,26,9)
macd.rename(columns={'close':'MACD'}, inplace=True)
stochastics = stochastics_oscillator(dfPrices,14)
stochastics.rename(columns={'close':'Stochastics'}, inplace=True)
atr = ATR(df,14)
atr.rename(columns={0:'ATR'}, inplace=True)
interpolated_df = interpolated_df.shift(-1)

In [19]:
final_data = pd.concat([df, macd,stochastics,atr], axis=1)
final_data.head()

,High,Low,close,Open,articles,H-L,H-PC,L-PC,MACD,Stochastics,ATR
3655,42.549999,41.880001,42.250000,41.443241,Terrorist Attack at Nightclub in Istanbul Kill...,0.669998,NaN,NaN,0.000000,NaN,0.669998
3654,42.549999,41.880001,42.250000,41.443241,. Shielding Seized Assets From Corruption’s Cl...,0.669998,0.299999,0.369999,0.000000,NaN,0.669998
3653,42.770000,41.730000,42.150002,41.345150,. Does Empathy Guide or Hinder Moral Action?. ...,1.040000,0.520000,0.520000,-0.001957,NaN,1.040000
3652,43.290001,42.570000,42.619999,41.806171,. When Finding the Right Lawyer Seems Daunting...,0.720001,1.139999,0.419998,0.009701,NaN,1.139999
3651,43.270000,42.950001,43.119999,42.296623,. Should the U.S. Embassy Be Moved From Tel Av...,0.319999,0.650001,0.330002,0.028591,NaN,0.650001


In [20]:
# Delete the entries with missing values (where the stochastics couldn't be computed yet) because have a lot of datapoints ;)
final_data = final_data.dropna()
final_data.head()

,High,Low,close,Open,articles,H-L,H-PC,L-PC,MACD,Stochastics,ATR
3642,43.090000,42.296666,42.790001,41.972928,". With Comic Book, Celebrities Pay Tribute to ...",0.793334,0.259998,0.533336,-0.005820,47.407404,0.793334
3641,43.320000,42.453334,42.750000,41.933692,. Trump’s Win Helps Carve a Path to Washington...,0.866666,0.529999,0.336667,-0.021609,44.444362,0.866666
3640,43.549999,42.610001,42.709999,41.894455,". Vine, the Six-Second Video App, Is Not Quite...",0.939998,0.799999,0.139999,-0.034015,41.481321,0.939998
3639,43.509998,42.889999,43.009998,42.188721,. Hispanic Surnames on the Rise in U.S. as Imm...,0.619999,0.799999,0.180000,-0.025271,44.318018,0.799999
3638,43.650002,42.119999,42.860001,42.041592,. Can Trump Get Tough With China?. The Oakland...,1.530003,0.640004,0.889999,-0.027276,18.987571,1.530003


In [21]:
del final_data['H-L']
del final_data['H-PC']
del final_data['L-PC']
del final_data['High']
del final_data['Low']
final_data.head()
# final_data.count()

,close,Open,articles,MACD,Stochastics,ATR
3642,42.790001,41.972928,". With Comic Book, Celebrities Pay Tribute to ...",-0.005820,47.407404,0.793334
3641,42.750000,41.933692,. Trump’s Win Helps Carve a Path to Washington...,-0.021609,44.444362,0.866666
3640,42.709999,41.894455,". Vine, the Six-Second Video App, Is Not Quite...",-0.034015,41.481321,0.939998
3639,43.009998,42.188721,. Hispanic Surnames on the Rise in U.S. as Imm...,-0.025271,44.318018,0.799999
3638,42.860001,42.041592,. Can Trump Get Tough With China?. The Oakland...,-0.027276,18.987571,1.530003


In [31]:
# final_data[::-1]
final_data = final_data[::-1]

In [32]:
# Save pandas frame in csv form

final_data.to_csv('Data/DataPrepared.csv',
                       sep=',', encoding='utf-8', index = False)
print("Saved successfully as DataPrepared.csv")


Saved successfully as DataPrepared.csv
